In [ ]:
import numpy as np

def generate_sphere_atoms(num_atoms, radius):
    points = []
    num_generated = 0
    while num_generated < num_atoms:
        phi = np.arccos(1 - 2 * np.random.rand())
        theta = 2 * np.pi * np.random.rand()
        x = radius * np.sin(phi) * np.cos(theta) + 100
        y = radius * np.sin(phi) * np.sin(theta) + 100
        z = radius * np.cos(phi) + 100
        if all(np.linalg.norm(np.array([x, y, z]) - np.array(point)) >= 1.0 for point in points):
            points.append((x, y, z))
            num_generated += 1

    x, y, z = zip(*points)
    return np.array(x), np.array(y), np.array(z)

def generate_random_point_inside_sphere(radius):
    # Generate random spherical coordinates
    theta = np.random.uniform(0, 2*np.pi)  # Azimuthal angle
    phi = np.arccos(np.random.uniform(-1, 1))  # Polar angle
    r = radius * np.random.uniform(0,1)

    # Convert spherical coordinates to Cartesian coordinates
    x = r * np.sin(phi) * np.cos(theta) + 100
    y = r * np.sin(phi) * np.sin(theta) + 100
    z = r * np.cos(phi) + 100

    return x, y, z

f = open('InitialFile.txt', 'w')
f.write('# Initialisation file of the particles\n\n')
n_monomers = 2000
n_nuclear_lamina = 1000
n_polymerases = 0
n_HP1 = 1000

n_chromatin_bonds = n_monomers - 1
n_poly_promoter_bonds = n_polymerases * 3

n_angles = 0
n_dihedral = 0

f.write(f'{n_monomers + n_nuclear_lamina + n_polymerases + n_HP1}  atoms\n')
f.write(f'{n_chromatin_bonds + n_poly_promoter_bonds}  bonds\n')
f.write(f'{n_angles}  angles\n')
f.write(f'{n_dihedral}  dihedrals\n\n')

f.write('10  atom types\n')
f.write('3  bond types\n')
f.write('0  angle types\n')
f.write('0  dihedral types\n\n')

f.write('0.0000   500.0000 xlo xhi \n0.0000   500.0000 ylo yhi \n0.0000   500.0000 zlo zhi\n\n')

f.write('Atom Type Labels\n\n')
f.write('1    A\n')
f.write('2    U\n')
f.write('3    M\n')
f.write('4    N\n')
f.write('5    P\n')
f.write('6    PU\n')
f.write('7    NA\n')
f.write('8    NM\n')
f.write('9    NU\n')
f.write('10    HP\n\n')

f.write('Bond Type Labels\n\n')
f.write('1    Normal\n')
f.write('2    MM\n')
f.write('3    PolyPromoter\n\n')

f.write('Masses\n\n')
f.write('1    1.00 \n')
f.write('2    1.00 \n')
f.write('3    1.00 \n')
f.write('4    1.00 \n')
f.write('5    1.00 \n')
f.write('6    1.00 \n')
f.write('7    1.00 \n')
f.write('8    1.00 \n')
f.write('9    1.00 \n')
f.write('10    1.00 \n\n')

monomer_types = np.random.choice(['A', 'U', 'M'], n_monomers)

f.write('Atoms\n\n')

molecule_number = 1

radius = 100.0
for i in range(n_monomers):
    x, y, z = generate_random_point_inside_sphere(radius)
    f.write(f'  {i + 1}  1  {monomer_types[i]}  {x}  {y}  {z}\n')

# Generate n_polymerases atoms
for i in range(n_polymerases):
    molecule_number = n_nuclear_lamina + i + 2
    x = np.random.uniform(100, 130)
    y = np.random.uniform(100, 130)
    z = np.random.uniform(100, 130)
    f.write(f'  {n_monomers + n_nuclear_lamina + i + 1}  {molecule_number}  P  {x}  {y}  {z}\n')
    
# Generate n_nuclear_lamina atoms on the surface of a sphere with radius 100 units
x, y, z = generate_sphere_atoms(n_nuclear_lamina, radius)

for i in range(n_nuclear_lamina):
    molecule_number =  i + 2
    
    f.write(f'  {n_monomers + n_polymerases + i + 1}  {molecule_number}  N  {x[i]}  {y[i]}  {z[i]}\n')

for i in range(n_HP1):
    molecule_number =  i + 2 + n_nuclear_lamina
    x, y, z = generate_random_point_inside_sphere(100)
    f.write(f'  {n_monomers + n_polymerases + n_nuclear_lamina + i + 1}  {molecule_number}  HP  {x}  {y}  {z}\n')


f.write('\nBonds\n\n')
for i in range(n_chromatin_bonds):
    if monomer_types[i] == 'M' and monomer_types[i + 1] == 'M':
        bond_type = 'MM'
    else:
        bond_type = 'Normal'
    f.write(f'  {i + 1}  {bond_type}  {i + 1}  {i + 2}\n')

num_bonds = n_chromatin_bonds
for i in range(n_polymerases):
    for j in [120,121,122]:
        num_bonds +=1
        f.write(f'  {num_bonds}  PolyPromoter  {j}  {n_monomers + n_nuclear_lamina + i + 1}\n')

f.close()
